In [1]:
import pandas as pd
import numpy as np

V druhom datasete budeme využívať tabuľku game, a game_goalie_stats.

Poslednych 5 zapasov medzi nimi. Statistiky za minulu sezonu a statistiky brankarov v minulej sezone.


In [3]:
gameInput = pd.read_csv("nhl-game-data/game.csv")
goalieInput = pd.read_csv("nhl-game-data/game_goalie_stats.csv")

In [8]:
goalieInput.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24646 entries, 0 to 24645
Data columns (total 19 columns):
game_id                       24646 non-null int64
player_id                     24646 non-null int64
team_id                       24646 non-null int64
timeOnIce                     24646 non-null int64
assists                       24646 non-null int64
goals                         24646 non-null int64
pim                           24646 non-null int64
shots                         24646 non-null int64
saves                         24646 non-null int64
powerPlaySaves                24646 non-null int64
shortHandedSaves              24646 non-null int64
evenSaves                     24646 non-null int64
shortHandedShotsAgainst       24646 non-null int64
evenShotsAgainst              24646 non-null int64
powerPlayShotsAgainst         24646 non-null int64
decision                      22868 non-null object
savePercentage                24607 non-null float64
powerPlaySavePercenta

Najskor si znova vyberieme len tie stĺpce ktoré potrebujeme.

In [4]:
gameInput1 = gameInput.loc[:,['game_id', 'season', 'date_time_GMT', 'home_team_id', 'away_team_id', 'date_time', 'away_goals', 'home_goals']]

DataFrame s brankármi si rovno aj upravíme. Vytvorime groupy podľa zápasov a následne podľa tímov. Potom sčítame, všetky hodnoty v skupinách. 

In [5]:
goalies = goalieInput.loc[:,['game_id','team_id','shots','saves']].groupby(['game_id','team_id']).sum().reset_index()

Tak môžeme vypočítať priemernú uspešnosť brankárov v danom zápase.

In [6]:
goalies['percentage'] = (goalies['saves'] / goalies['shots'])
print(type(goalies.loc[goalies['game_id'] == 2010020001]['percentage'].values[0]))

<class 'numpy.float64'>


Následne pre každý zápas zistíme úspešnosť brankárov.

In [7]:
goalies1 = pd.DataFrame(columns=['game_id', 'home_percentage', 'away_percentage'])
for index, row in gameInput1.iterrows():
    if not goalies.loc[(goalies['game_id'] == row['game_id']) & (goalies['team_id'] == row['away_team_id'])].empty:
        awayPerc = goalies.loc[(goalies['game_id'] == row['game_id']) & (goalies['team_id'] == row['away_team_id'])]
    if not goalies.loc[(goalies['game_id'] == row['game_id']) & (goalies['team_id'] == row['home_team_id'])].empty:
        homePerc = goalies.loc[(goalies['game_id'] == row['game_id']) & (goalies['team_id'] == row['home_team_id'])]
    d = {'game_id': row['game_id'], 'home_percentage': homePerc['percentage'].values[0], 'away_percentage': awayPerc['percentage'].values[0]}
    goalies1.at[index, :] = d
goalies1

,game_id,home_percentage,away_percentage
0,2.01103e+09,0.884615,0.888889
1,2.01103e+09,0.911765,0.95
2,2.01103e+09,0.892857,0.870968
3,2.01103e+09,0.909091,0.928571
4,2.01103e+09,0.9,0.964286
...,...,...,...
11429,2.01803e+09,0.73913,0.931034
11430,2.01803e+09,0.913043,0.918919
11431,2.01803e+09,0.904762,0.974359
11432,2.01803e+09,0.870968,0.965517


Teraz spojíme tabuľky game a goalies.

In [8]:
gameInput1 = gameInput1.join(goalies1, lsuffix='', rsuffix='_goalie')

In [34]:
gameInput1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11434 entries, 0 to 11433
Data columns (total 11 columns):
game_id            11434 non-null int64
season             11434 non-null int64
date_time_GMT      11434 non-null object
home_team_id       11434 non-null int64
away_team_id       11434 non-null int64
date_time          11434 non-null object
away_goals         11434 non-null int64
home_goals         11434 non-null int64
game_id_goalie     11434 non-null object
home_percentage    11434 non-null object
away_percentage    11434 non-null object
dtypes: int64(6), object(5)
memory usage: 982.7+ KB


Teraz si upravíme úspešnosť brankárov na numerickú hodnotu, aby sme mohli počítať priemer aj z tejto hodnoty.

In [9]:
gameAwayValues = gameInput1.assign(away_percentage=pd.to_numeric(gameInput1['away_percentage'], errors='coerce'))\
    .loc[:,['away_team_id', 'away_goals', 'home_goals', 'away_percentage', 'season']].groupby(['away_team_id', 'season'])\
    .mean().reset_index()
gameHomeValues = gameInput1.assign(home_percentage=pd.to_numeric(gameInput1['home_percentage'], errors='coerce'))\
    .loc[:,['home_team_id', 'away_goals', 'home_goals', 'home_percentage', 'season']].groupby(['home_team_id', 'season'])\
    .mean().reset_index()
# gameAwayValues
gameHomeValues


,home_team_id,season,away_goals,home_goals,home_percentage
0,1,20102011,2.463415,2.292683,0.898303
1,1,20112012,2.461538,2.846154,0.906777
2,1,20122013,2.458333,2.750000,0.892649
3,1,20132014,2.243902,2.560976,0.916448
4,1,20142015,2.195122,2.219512,0.931411
...,...,...,...,...,...
267,53,20162017,2.853659,2.536585,0.918366
268,53,20172018,2.878049,2.487805,0.914878
269,53,20182019,2.463415,2.707317,0.922345
270,54,20172018,2.470588,3.588235,0.917683


Vytvoríme si dataFrame s potrebnými atribútmi.

In [10]:
dataSet1 = pd.DataFrame(columns=['game_id','season', 'date_time', 'game1home','game1away', 'game1isHome','game2home', \
                                 'game2away','game2isHome','game3home','game3away','game3isHome','game4home','game4away',\
                                 'game4isHome','game5home','game5away','game5isHome', 'home_perc', 'away_perc','home_team_id', 'away_team_id', \
                                 'home_scored_avg', 'home_conceded_avg', 'away_scored_avg', 'away_conceded_avg', \
                                 'total'])

Nasledujúci kód je plne prispôsobený nášmu riešeniu, a preto sa môže zdať, že je neprehľadný. 

Pre každý zápas najskôr nájdeme priemerné počty gólov v domácich alebo hosťujúcich zápasoch v minulých sezónach. S týmti zároveň nájdeme aj percentuálnu úspešnosť brankárov v týchto sezónach. Vytvoríme si objekt.  

Následne nájdeme pre tieto dva tímy posledných 5 spoločných zápasov, a z nich vyberieme počet gólov domácich a hostí. Dynamicky ich priradíme do objektu d. Zároveň, domáci tím nemusí byť v predchádzajúcich zápasoch ako domáci tím, tak musíme zaznamenať, či bol ako domáci tím aj v týchto zápasoch. Pri každom zápase je flag gameXisHome. Nakoniec ich uložíme do dataSet1.

In [11]:
gameAwayValues = gameAwayValues.dropna()
gameHomeValues = gameHomeValues.dropna()
for index, row in gameInput1.iterrows():
    season = row['season'] - 10001
    if not gameAwayValues.loc[(gameAwayValues['away_team_id'] == row['away_team_id']) & (gameAwayValues['season'] == season)].empty:
        if not gameHomeValues.loc[(gameHomeValues['home_team_id'] == row['home_team_id']) & (gameHomeValues['season'] == season)].empty:
            awayTeam = gameAwayValues.loc[(gameAwayValues['away_team_id'] == row['away_team_id']) & (gameAwayValues['season'] == season)]
            homeTeam = gameHomeValues.loc[(gameHomeValues['home_team_id'] == row['home_team_id']) & (gameHomeValues['season'] == season)]
#             print(homeTeam, awayTeam)
            d = {'game_id': row['game_id'],\
                 'season': row['season'],\
                 'date_time': row['date_time'],\
                 'home_team_id': row['home_team_id'], \
                 'home_perc': homeTeam['home_percentage'].values[0], \
                 'away_perc': awayTeam['away_percentage'].values[0], \
                 'home_scored_avg': homeTeam['home_goals'].values[0], \
                 'home_conceded_avg': homeTeam['away_goals'].values[0], \
                 'away_team_id': row['away_team_id'], \
                 'away_scored_avg': awayTeam['away_goals'].values[0], \
                 'away_conceded_avg': awayTeam['home_goals'].values[0], \
                 'total': row['home_goals'] + row['away_goals']}
            for i in range(1,6):
                d['game' + str(i) + 'home'] = 0
                d['game' + str(i) + 'away'] = 0
                d['game' + str(i) + 'isHome'] = 1
            if not gameInput.loc[((gameInput['date_time_GMT'] < row['date_time_GMT']) & \
                                  ((gameInput['home_team_id'] == row['home_team_id']) & \
                                  (gameInput['away_team_id'] == row['away_team_id'])) | \
                                  ((gameInput['home_team_id'] == row['away_team_id']) & \
                                  (gameInput['away_team_id'] == row['home_team_id'])))].tail(5).empty:
                previousGames = gameInput.loc[((gameInput['date_time_GMT'] < row['date_time_GMT']) & \
                                  ((gameInput['home_team_id'] == row['home_team_id']) & \
                                  (gameInput['away_team_id'] == row['away_team_id'])) | \
                                  ((gameInput['home_team_id'] == row['away_team_id']) & \
                                  (gameInput['away_team_id'] == row['home_team_id'])))].tail(5)
                i = 1
                for index1, row1 in previousGames.iterrows():
                    if (row1.home_team_id == row.home_team_id):
                        d['game' + str(i) + 'home'] = row1['home_goals']
                        d['game' + str(i) + 'away'] = row1['away_goals']
                        d['game' + str(i) + 'isHome'] = 1
                    else:
                        d['game' + str(i) + 'home'] = row1['away_goals']
                        d['game' + str(i) + 'away'] = row1['home_goals']
                        d['game' + str(i) + 'isHome'] = 0
                    i += 1
                    
            dataSet1.at[index, :] = d

In [16]:
dataSet2 = dataSet1.sample(frac=1).reset_index(drop=True)
dataSet2

,game_id,season,date_time,game1home,game1away,game1isHome,game2home,game2away,game2isHome,game3home,...,game5isHome,home_perc,away_perc,home_team_id,away_team_id,home_scored_avg,home_conceded_avg,away_scored_avg,away_conceded_avg,total
0,2016020472,20162017,2016-12-18,4,1,1,3,2,1,4,...,0,0.912661,0.911323,23,29,2.34146,2.87805,2.41463,3.02439,7
1,2016030114,20162017,2017-04-18,0,4,0,6,3,1,1,...,1,0.913061,0.895157,3,8,3.04651,2.53488,2.82927,3.2439,3
2,2014020254,20142015,2014-11-16,2,1,0,1,6,0,1,...,0,0.916448,0.91726,1,21,2.56098,2.2439,2.77273,2.79545,5
3,2013020508,20132014,2013-12-17,2,3,1,3,0,1,5,...,0,0.90422,0.902351,21,25,2.70833,2.83333,2.70833,2.83333,8
4,2013030156,20132014,2014-04-29,2,4,1,3,1,0,1,...,1,0.905179,0.89838,30,21,2.19231,2.38462,2.125,3.5,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9844,2015020982,20152016,2016-03-06,3,4,0,2,6,0,2,...,1,0.923938,0.898703,3,2,2.76923,2.34615,2.77778,2.88889,10
9845,2018020079,20182019,2018-10-16,6,2,0,1,6,0,1,...,0,0.911178,0.897113,3,21,3.07317,3.09756,2.70455,3.40909,5
9846,2012020043,20122013,2013-01-25,4,3,0,2,3,0,3,...,1,0.914981,0.908849,12,7,2.58537,2.68293,2.31707,2.92683,9
9847,2014020851,20142015,2015-02-18,5,3,1,1,0,1,4,...,0,0.909961,0.897096,19,25,3.38636,2.40909,2.59091,3.18182,5


Uložíme si tento dataFrame.

In [17]:
dataSet2.to_csv('.\datasets\dataFrameGameGoalies.csv', encoding='utf-8', index=False)

Vymažeme nepotrebné stĺpce.

In [18]:
dataSetFinal = dataSet2.drop('date_time',1)

Teraz si rozdelíme dataFrame na 2 množiny. Znova podľa sezón.

In [19]:
dataTrain1 = dataSetFinal.loc[0:int(len(dataSetFinal)*0.8)]
dataTest1 = dataSetFinal.loc[int(len(dataSetFinal)*0.8) + 1:]

Následne tieto množiny rozdelíme na vstupné vektory a očakávané hodnoty. 

In [20]:
dataSet2TrainX = dataTrain1.drop('season',1).drop('game_id',1).drop('total',1)
dataSet2TrainY = dataTrain1.total
dataSet2TestX = dataTest1.drop('season',1).drop('game_id',1).drop('total',1)
dataSet2TestY = dataTest1.total

In [21]:
dataSet2TrainX

,game1home,game1away,game1isHome,game2home,game2away,game2isHome,game3home,game3away,game3isHome,game4home,...,game5away,game5isHome,home_perc,away_perc,home_team_id,away_team_id,home_scored_avg,home_conceded_avg,away_scored_avg,away_conceded_avg
0,4,1,1,3,2,1,4,1,0,3,...,3,0,0.912661,0.911323,23,29,2.34146,2.87805,2.41463,3.02439
1,0,4,0,6,3,1,1,4,0,5,...,2,1,0.913061,0.895157,3,8,3.04651,2.53488,2.82927,3.2439
2,2,1,0,1,6,0,1,0,1,2,...,3,0,0.916448,0.91726,1,21,2.56098,2.2439,2.77273,2.79545
3,2,3,1,3,0,1,5,0,1,2,...,1,0,0.90422,0.902351,21,25,2.70833,2.83333,2.70833,2.83333
4,2,4,1,3,1,0,1,0,1,4,...,4,1,0.905179,0.89838,30,21,2.19231,2.38462,2.125,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7875,4,2,0,1,4,0,3,4,0,3,...,5,0,0.905767,0.913435,17,2,2.56098,2.68293,2.7561,3
7876,2,5,0,2,4,0,5,1,1,1,...,3,0,0.899588,0.913435,26,2,2.80488,2.58537,2.7561,3
7877,4,1,1,2,3,0,5,4,1,2,...,5,0,0.922163,0.907136,16,18,3.02326,2.39535,2.59615,2.92308
7878,2,1,1,3,0,0,2,1,0,1,...,3,1,0.92209,0.907577,29,1,2.81818,2.52273,2.97727,3.25


In [62]:
dataSet2TrainY

11049    2
9192     3
10104    5
11113    6
10684    5
        ..
633      6
634      6
635      5
636      3
637      4
Name: total, Length: 8491, dtype: object

Tieto množiny uložíme do .csv súborov.

In [22]:
dataSet2TrainX.to_csv('.\datasets\dataSet2TrainX.csv', encoding='utf-8', index=False)
dataSet2TrainY.to_csv('.\datasets\dataSet2TrainY.csv', encoding='utf-8',index=False)
dataSet2TestX.to_csv('.\datasets\dataSet2TestX.csv', encoding='utf-8', index=False)
dataSet2TestY.to_csv('.\datasets\dataSet2TestY.csv', encoding='utf-8', index=False)

C:\Users\Filip\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  
C:\Users\Filip\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  after removing the cwd from sys.path.


In [30]:
print(len(dataTrain1.loc[dataTrain1.total == 6])/len(dataTrain1))
print(len(dataTrain1.loc[dataTrain1.total == 5])/len(dataTrain1))

0.10545685279187818
0.24048223350253808


In [33]:
print(len(dataTest1.loc[dataTest1.total == 5])/len(dataTest1))
print(len(dataTest1.loc[dataTest1.total == 6])/len(dataTest1))

0.2656170644997461
0.10208227526663281
